In [474]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import requests
from bs4 import BeautifulSoup

In [456]:
# send request
URL=[]
pages=['1','51','101','151','201']

for p in pages:
    l='https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start='+p
    URL.append(l)
page=[]
for i in URL:
    page.append(requests.get(i))
soup=[]
for i in page:
    soup.append(BeautifulSoup(i.text,'html.parser'))


In [457]:
movie_name=[]
ratings=[]
id=[]
genre=[]
votes=[]
gross=[]
year=[]
metascores=[]
for i in soup:
    movie=i.find('div',{'class':'lister-list'}).find_all(class_='lister-item mode-advanced')
    for item in movie:
        #title of the movie
        title=item.find(class_='lister-item-header').find('a').get_text()
        movie_name.append(title)
        #ratings of the movie
        rate=item.find(class_='inline-block ratings-imdb-rating').find('strong').get_text()
        ratings.append(rate)
        #id of the movie
        raw_id=item.find(class_='lister-item-index unbold text-primary').get_text()
        id.append(raw_id)
        #genre of the movie
        genre_list=item.find(class_='genre').get_text().replace('\n','')
        genre.append(genre_list)
        #votes of the movie
        vote=item.find('span',attrs={'name':'nv'}).get_text()
        votes.append(vote)
        #gross of the movie
        gross_list=item.find(class_='sort-num_votes-visible').get_text().split('|')[-1].replace('\n','')
        gross.append(gross_list)
        #year of the movie
        year_list=item.find(class_='lister-item-year text-muted unbold').get_text()
        year.append(year_list)
        #metascores of the movie
        if item.find('div',class_='ratings-metascore') is not None:
            m_score=item.find('span',class_='metascore').text
            metascores.append(int(m_score))
        else:
            metascores.append(None)

In [460]:
#description of the movie

summary=[]
description_text=[]
description=[]
for i in soup:
    summary_text=i.find_all('p',{'class':'text-muted'})
    for item in summary_text:
        summary.append(item.text)
for i in range(len(summary)):
    if(int(i)%2!=0):
        description_text.append(summary[i])
for i in range(len(description_text)):
    j=description_text[i].split('\n')[-1]
    description.append(j)
    

In [461]:
#reading actor director name

p=[]
a=[]
director=[]
actor=[]
for i in soup:
    director_actor=i.find_all('p',{'class':''})
    for item in director_actor:
        p.append(item.text)
for i in range(len(p)):
    if(int(i)%2!=0):
        a.append(p[i])
for i in range(len(a)):
    q=a[i].split('|')
    director_list=q[0].replace('\n','').split(':')[-1]
    actor_list=q[-1].replace('\n','').split(':')[-1]
    director.append(director_list)
    actor.append(actor_list)


In [482]:
#creating dataframe

array=[]
for i,ti,ye,ge,ra,me,de,di,st,vo,gr in zip(id,movie_name,year,genre,ratings,metascores,description,director,actor,votes,gross):
    array.append({'ID':i, 'TITLE':ti, 'RELEASING YEAR':ye, 'GENRE':ge, 'RATINGS':ra, 'METASCORES':me, 'DESCRIPTION':de, 'DIRECTOR':di, 'STARS':st, 'VOTES':vo, 'GROSS':gr})

In [483]:
data=pd.DataFrame(array)

In [485]:
data=data[['ID', 'TITLE', 'RELEASING YEAR', 'GENRE', 'RATINGS', 'METASCORES', 'DESCRIPTION', 'DIRECTOR', 'STARS', 'VOTES','GROSS']]

In [489]:
data

,ID,TITLE,RELEASING YEAR,GENRE,RATINGS,METASCORES,DESCRIPTION,DIRECTOR,STARS,VOTES,GROSS
0,1.,The Shawshank Redemption,(1994),Drama,9.3,80.0,Two imprisoned men bond over a number of y...,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...","2,184,067",Gross:$28.34M
1,2.,The Godfather,(1972),"Crime, Drama",9.2,100.0,The aging patriarch of an organized crime ...,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...","1,504,636",Gross:$134.97M
2,3.,The Dark Knight,(2008),"Action, Crime, Drama",9.0,84.0,When the menace known as the Joker wreaks ...,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","2,167,140",Gross:$534.86M
3,4.,The Godfather: Part II,(1974),"Crime, Drama",9.0,90.0,The early life and career of Vito Corleone...,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall, Dian...","1,053,696",Gross:$57.30M
4,5.,The Lord of the Rings: The Return of the King,(2003),"Adventure, Drama, Fantasy",8.9,94.0,Gandalf and Aragorn lead the World of Men ...,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...","1,553,736",Gross:$377.85M
5,6.,Pulp Fiction,(1994),"Crime, Drama",8.9,94.0,"The lives of two mob hitmen, a boxer, a ga...",Quentin Tarantino,"John Travolta, Uma Thurman, Samuel L. Jackson,...","1,715,742",Gross:$107.93M
6,7.,Schindler's List,(1993),"Biography, Drama, History",8.9,94.0,In German-occupied Poland during World War...,Steven Spielberg,"Liam Neeson, Ralph Fiennes, Ben Kingsley, Caro...","1,140,041",Gross:$96.90M
7,8.,12 Angry Men,(1957),Drama,8.9,96.0,A jury holdout attempts to prevent a misca...,Sidney Lumet,"Henry Fonda, Lee J. Cobb, Martin Balsam, John ...","636,366",Gross:$4.36M
8,9.,Inception,(2010),"Action, Adventure, Sci-Fi",8.8,74.0,A thief who steals corporate secrets throu...,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen...","1,915,927",Gross:$292.58M
9,10.,Fight Club,(1999),Drama,8.8,66.0,An insomniac office worker and a devil-may...,David Fincher,"Brad Pitt, Edward Norton, Meat Loaf, Zach Grenier","1,744,504",Gross:$37.03M
